In [35]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.data.materialLibrary import MATLIB
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.functions.hexLattice import hexLat
from serpentGenerator.functions.pinStack import pinStack
from serpentGenerator.functions.surfs import surfs
from serpentGenerator.functions.surf import surf
from serpentGenerator.functions.cell import cell
from serpentGenerator.functions.cells import cells
from serpentGenerator.functions.lats import lats
from serpentGenerator.functions.mix import mix
from serpentGenerator.functions.mats import mats
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.core import core
from serpentGenerator.functions.housing import housing
from serpentGenerator.functions.universe import universe
from serpentGenerator.functions.branches import branches

In [36]:
fuel1 = MATLIB['UO2'].duplicateMat("fuel1")
fuel1.set('fractions', np.array([-0.04319337211, -0.83830401789, -.11850261])) # 4.9 % wt enrichment
fuel1.set('dens', -10.5216)
fuel1.set('rgb', "238 59 49")

fuel2 = fuel1.duplicateMat("fuel2")
fuel2.set('rgb', "255 165 0")

fuel3 = fuel1.duplicateMat("fuel3")
fuel3.set('rgb', "255 255 0")

fuel4 = fuel1.duplicateMat("fuel4")
fuel4.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261])) # 2.0 % wt enrichment 
fuel4.set('rgb', "255 255 255")

mod = MATLIB['H2O'].duplicateMat("water")
mod.set('rgb', "198 226 255")
mod.set('dens', -7.200)

clad = MATLIB['Zr'].duplicateMat("clad")
clad.set('rgb', "183 183 183")

ref = MATLIB['Reflector'].duplicateMat("reflector") # reflector plenum
ref.set('rgb', "204 153 255")

ba = MATLIB['B'].duplicateMat("B4C") # 90% wt enriched b-10

MB = 10.8
MC = 12
MB4C = MB*4 + MC
rhoB4C = 2.52
AvroNorm = .6022
nb4c = rhoB4C*AvroNorm/MB4C
nc12 = nb4c
b10_enrich = .90
b11_enrich = 1 - b10_enrich
nb10 = nb4c*b10_enrich
nb11 = nb4c*b11_enrich

b4cdens = -2.52
ba.set('rgb', "255 255 0")
ba.set('nuclides', np.array([5010, 5011, 6012]))
ba.set('fractions', np.array([nb10, nb11, nc12]))
ba.set('dens', -1*rhoB4C)
ba.set('isBurn', True)

matList = mats()
matList.addMats([fuel1, fuel2, fuel3, fuel4,  mod, clad, ref, ba])

# print(matList.toString())

# Axial Layers

1. Pin pitch (cm) 1.2623
2. UO2 pellet radius (cm) 0.4096
3. Zircaloy clad inner radius (cm) 0.4187
4. Zircaloy clad outer radius (cm) 0.4760
5. Guide thimble and instrumentation tube
6. Zircaloy inner radius (cm) 0.5531
7. Zircaloy outer radius (cm) 0.6133

In [37]:
ro_fuel = .4096
ro_clad = .4760
ri_gt = .5531
ro_gt = .6133

In [38]:
F1 = pin("F1", 3)
F1.set('materials', np.array([fuel1, clad, mod]))
F1.set('radii', np.array([ro_fuel, ro_clad]))

F2 = F1.duplicatePin("F2")
F2.set('materials', np.array([fuel2, clad, mod]))

F3 = F1.duplicatePin("F3")
F3.set('materials', np.array([fuel3, clad, mod]))

F4 = F1.duplicatePin("F4")
F4.set('materials', np.array([fuel4, clad, mod]))

G1 = pin("G1", 3)
G1.set('materials', np.array([mod, clad, mod]))
G1.set('radii', np.array([ri_gt, ro_gt]))

W1 = pin("W1", 3)
W1.set('materials', np.array([mod]))

RT = pin("RT", 1) #top reflector
RT.set('materials', np.array([ref]))

RB =  RT.duplicatePin("RB")

pinList1 = pins()
pinList1.addPins([F1, F2, F3, F4, W1, RT, RB, G1])

# Explicit BigT GuideThimble Cells (Pins)

In [39]:
    riClad = .5531
    roClad = .6133

    tClad = roClad - riClad

    span1 = 70

    gap1 = (360-span1*4)/4
    theto1 = (90 - span1)/2
    thetf1 = 360 - theto1

    span2 = 55

    gap2 = (360-span2*4)/4
    theto2 = (90 - span2)/2
    thetf2 = 360 - theto2

    span3 = 60

    gap3 = (360-span3*4)/4
    theto3 = (90 - span3)/2
    thetf3 = 360 - theto3

    t1 = .090/10
    t2 = .089/10
    t3 = .019/10

    riPad1 = riClad+ (tClad - t1)/2 
    roPad1 = riPad1 + t1

    riPad2 = riClad+ (tClad - t2)/2 
    roPad2 = riPad2 + t2

    riPad3 = riClad+ (tClad - t3)/2 
    roPad3 = riPad3 + t3

    # vol1 = 4*(span1/360)*np.pi*(roPad1*roPad1 - riPad1*riPad1)*200
    # print(vol1)

    # vol2 = 4*(span2/360)*np.pi*(roPad2*roPad2 - riPad2*riPad2)*200
    # print(vol2)

    # vol3 = 4*(span3/360)*np.pi*(roPad3*roPad3 - riPad3*riPad3)*200
    # print(vol3)

    # totvol = vol1*9*24 + vol2*12*24 + vol3*16*24
    # print(totvol)

    r1 = [riClad, riPad1, roPad1, roClad]
    r2 = [riClad, riPad2, roPad2, roClad]
    r3 = [riClad, riPad3, roPad3, roClad]

    crs1 = [None]*len(r1)
    baps1 = [None]*len(r1)
    bapscc1 = [None]*len(r1)
    clps1 = [None]*len(r1)
    clpscc1 = [None]*len(r1)

    crs2= [None]*len(r2)
    baps2= [None]*len(r2)
    bapscc2 = [None]*len(r2)
    clps2= [None]*len(r2)
    clpscc2= [None]*len(r2)

    crs3= [None]*len(r3)
    baps3= [None]*len(r3)
    bapscc3 = [None]*len(r3)
    clps3= [None]*len(r3)
    clpscc3= [None]*len(r3)


    alpha11= theto1
    alpha12 = alpha11 + span1
    alpha13 = thetf1
    alpha14 = (alpha13 + gap1) % 360
                        
    alpha21 = theto2
    alpha22 = alpha21 + span2
    alpha23 = thetf2
    alpha24 = (alpha23 + gap2) % 360

    alpha31 = theto3
    alpha32 = alpha31 + span3
    alpha33 = thetf3
    alpha34 = (alpha33 + gap3) % 360

    ind3 = 0 
    burnmat = ba

    for i in range(0, len(r1)):
        crs1[i] = surf("crt1"+str(i+1)+str(ind3+1), "cyl", np.array([0.0, 0.0, r1[i]]))
        baps1[i] = surf("bat1"+str(i+1)+str(ind3+1), "pad", np.array([0.0, 0.0, riPad1, roPad1, alpha11, alpha12]))
        clps1[i] = surf("clt1"+str(i+1)+str(ind3+1), "pad", np.array([0.0, 0.0, riPad1, roPad1, alpha13, alpha14]))
        bapscc1[i] = cell("bapadt1"+str(i+1)+str(ind3+1), np.array([baps1[i]]), np.array([1]), burnmat.id)
        clpscc1[i] = cell("clpadt1"+str(i+1)+str(ind3+1), np.array([clps1[i]]), np.array([1]), clad.id)
        alpha11 = alpha12 + gap1
        alpha12 = alpha11 + span1
        alpha13 = alpha14 + span1
        alpha14 = alpha13 + gap1
     
    cc11 = cell("cc11"+str(ind3+1), np.array([crs1[0]]), np.array([1]), mod.id)
    cc12 = cell("cc12"+str(ind3+1), np.array([crs1[0], crs1[1]]), np.array([0, 1]), clad.id)
    cc13 = cell("cc13"+str(ind3+1), np.array([crs1[2], crs1[3]]), np.array([0, 1]), clad.id)
    cc14 = cell("cc14"+str(ind3+1), np.array([crs1[3]]), np.array([0]), mod.id)


    surfList1 = surfs()
    surfList1.addSurfs(crs1)
    surfList1.addSurfs(baps1)
    surfList1.addSurfs(clps1)

    cellList1 = cells()
    cellList1.addCells(bapscc1)
    cellList1.addCells(clpscc1)
    cellList1.addCells([cc11, cc12, cc13, cc14])

    T1 = universe("T1"+str(ind3+1))
    T1.surfs = surfList1
    T1.cells = cellList1

    for i in range(0, len(r2)):
        crs2[i] = surf("crt2"+str(i+1)+str(ind3+1), "cyl", np.array([0.0, 0.0, r2[i]]))
        baps2[i] = surf("bat2"+str(i+1)+str(ind3+1), "pad", np.array([0.0, 0.0, riPad2, roPad2, alpha21, alpha22]))
        clps2[i] = surf("clt2"+str(i+1)+str(ind3+1), "pad", np.array([0.0, 0.0, riPad2, roPad2, alpha23, alpha24]))
        bapscc2[i] = cell("bapadt2"+str(i+1)+str(ind3+1), np.array([baps2[i]]), np.array([1]), burnmat.id)
        clpscc2[i] = cell("clpadt2"+str(i+1)+str(ind3+1), np.array([clps2[i]]), np.array([1]), clad.id)
        alpha21 = alpha22 + gap2
        alpha22 = alpha21 + span2
        alpha23 = alpha24 + span2
        alpha24 = alpha23 + gap2

    cc21 = cell("cc21"+str(ind3+1), np.array([crs2[0]]), np.array([1]), mod.id)
    cc22 = cell("cc22"+str(ind3+1), np.array([crs2[0], crs2[1]]), np.array([0, 1]), clad.id)
    cc23 = cell("cc23"+str(ind3+1), np.array([crs2[2], crs2[3]]), np.array([0, 1]), clad.id)
    cc24 = cell("cc24"+str(ind3+1), np.array([crs2[3]]), np.array([0]), mod.id)

    surfList2 = surfs()
    surfList2.addSurfs(crs2)
    surfList2.addSurfs(baps2)
    surfList2.addSurfs(clps2)

    cellList2 = cells()
    cellList2.addCells(bapscc2)
    cellList2.addCells(clpscc2)
    cellList2.addCells([cc21, cc22, cc23, cc24])

    T2 = universe("T2"+str(ind3+1))
    T2.surfs = surfList2
    T2.cells = cellList2

    for i in range(0, len(r3)):
        crs3[i] = surf("crt3"+str(i+1)+str(ind3+1), "cyl", np.array([0.0, 0.0, r3[i]]))
        baps3[i] = surf("bat3"+str(i+1)+str(ind3+1), "pad", np.array([0.0, 0.0, riPad3, roPad3, alpha31, alpha32]))
        clps3[i] = surf("clt3"+str(i+1)+str(ind3+1), "pad", np.array([0.0, 0.0, riPad3, roPad3, alpha33, alpha34]))
        bapscc3[i] = cell("bapadt3"+str(i+1)+str(ind3+1), np.array([baps3[i]]), np.array([1]), burnmat.id)
        clpscc3[i] = cell("clpadt3"+str(i+1)+str(ind3+1), np.array([clps3[i]]), np.array([1]), clad.id)
        alpha31 = alpha32 + gap3
        alpha32 = alpha31 + span3
        alpha33 = alpha34 + span3
        alpha34 = alpha33 + gap3


    cc31 = cell("cc31"+str(ind3+1), np.array([crs3[0]]), np.array([1]), mod.id)
    cc32 = cell("cc32"+str(ind3+1), np.array([crs3[0], crs3[1]]), np.array([0, 1]), clad.id)
    cc33 = cell("cc33"+str(ind3+1), np.array([crs3[2], crs3[3]]), np.array([0, 1]), clad.id)
    cc34 = cell("cc34"+str(ind3+1), np.array([crs3[3]]), np.array([0]), mod.id)

    surfList3 = surfs()
    surfList3.addSurfs(crs3)
    surfList3.addSurfs(baps3)
    surfList3.addSurfs(clps3)

    cellList3 = cells()
    cellList3.addCells(bapscc3)
    cellList3.addCells(clpscc3)
    cellList3.addCells([cc31, cc32, cc33, cc34])

    T3 = universe("T3"+str(ind3+1))
    T3.surfs = surfList3
    T3.cells = cellList3

# Channels
#### Fuel Rods 5 Axial layers
1. Bottom Reflector Span 20 cm
2. Bottom Axial Blanket Span 5 cm 
3. 4.95% Enriched Fuel Axial Span 185 cm
4. Top Axial Blanket Span 10 cm
5. Top Refelector Span 20 cm

#### Guide Tube 5 Axial layers
1. Bottom Reflector Span 20 cm
2. Bottom BA Cutoff Span 5 cm
3. BA integration Span 185 cm
4. Top BA Cutoff Span 10 cm
5. Top Refelector Span 20 cm

#### Instrumentation Tube 3 Axial Layers
1. Bottom Reflector Span 20 cm
2. Guide Tube 200 cm
5. Top Refelector Span 20 cm

#### Water Gap 3 Axial Layers
1. Bottom Reflector Span 20 cm
2. Water 200 cm
5. Top Refelector Span 20 cm


In [40]:
FC1 = pinStack("FC1", 0, 0, 5)
FC1.setStack(np.array([RB, F4, F1, F4, RT]), np.array([0.0, 20.0, 25.0, 210.0, 220.0]))

FC2 = FC1.duplicateLat("FC2")
FC2.replacePin(F1, F2)

FC3 = FC1.duplicateLat("FC3")
FC3.replacePin(F1, F3)

GT1 = pinStack("GT1", 0, 0, 5)
GT1.setStack(np.array([RB, G1, T1, G1, RT]), np.array([0.0, 20.0, 25.0, 210.0, 220.0]))

GT2 = GT1.duplicateLat("GT2")
GT2.replacePin(T1, T2)

GT3 = GT1.duplicateLat("GT3")
GT3.replacePin(T1, T3)

CT1 = pinStack("CT1", 0, 0, 3)
CT1.setStack(np.array([RB, G1, RT]), np.array([0.0, 20.0, 220.0]))

WG1 = CT1.duplicateLat("WG1")
WG1.replacePin(G1, W1)
               
latList1 = lats()
latList1.addLats([FC1, FC2, FC3, GT1, GT2, GT3, CT1, WG1])

latlistRef = lats()
latlistRef.addLats([FC1, CT1, WG1])


# 3D Assemblys
1. Pin pitch (cm) 1.2623
2. 17 x 17

In [41]:
FA1 = sqLat("FA1", 0, 0, 19, 1.2623)

FA1Map = np.array([
    [WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, CT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, GT1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, FC1, WG1],
    [WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1, WG1]
])

FA1.setMap(FA1Map)

FA1_REF = FA1.duplicateLat("FA1")
FA1_REF.replacePin(GT1, CT1)

FA2 = FA1.duplicateLat("FA2")
FA2.replacePin(FC1, FC2)
FA2.replacePin(GT1, GT2)

FA3 = FA1.duplicateLat("FA3")
FA3.replacePin(FC1, FC3)
FA3.replacePin(GT1, GT3)

WA1 = FA1.duplicateLat("WA1")
WA1.replacePin(FC1, WG1)
WA1.replacePin(GT1, WG1)
WA1.replacePin(CT1, WG1)

latList1.addLats([FA1, FA2, FA3, WA1])

latlistRef.addLats([FA1_REF, WA1])

# print(latList.toString())

# Core Structure
1. Assembly Pitch 21.6038
2. N FA1 = 9
3. N FA2 = 12
4. N FA3 = 16

In [42]:
CORE1 = sqLat("CORE1", 0, 0, 11, 21.6038)

CORE1Map = np.array([
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, FA3, FA3, FA3, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, FA3, FA2, FA2, FA2, FA3, WA1, WA1, WA1],
[WA1, WA1, FA3, FA2, FA1, FA1, FA1, FA2, FA3, WA1, WA1],
[WA1, WA1, FA3, FA2, FA1, FA1, FA1, FA2, FA3, WA1, WA1],
[WA1, WA1, FA3, FA2, FA1, FA1, FA1, FA2, FA3, WA1, WA1],
[WA1, WA1, WA1, FA3, FA2, FA2, FA2, FA3, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, FA3, FA3, FA3, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1],
[WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1, WA1]
])

CORE1.setMap(CORE1Map)

CORE_REF = CORE1.duplicateLat("REFCORE")
CORE_REF.replacePin(FA1, FA1_REF)
CORE_REF.replacePin(FA2, FA1_REF)
CORE_REF.replacePin(FA3, FA1_REF)


print(CORE1.toString())
print(CORE_REF.toString())



lat CORE1 1 0 0  11 11 21.6038
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 FA3 FA3 FA3 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 FA3 FA2 FA2 FA2 FA3 WA1 WA1 WA1 
WA1 WA1 FA3 FA2 FA1 FA1 FA1 FA2 FA3 WA1 WA1 
WA1 WA1 FA3 FA2 FA1 FA1 FA1 FA2 FA3 WA1 WA1 
WA1 WA1 FA3 FA2 FA1 FA1 FA1 FA2 FA3 WA1 WA1 
WA1 WA1 WA1 FA3 FA2 FA2 FA2 FA3 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 FA3 FA3 FA3 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 


lat REFCORE 1 0 0  11 11 21.6038
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 FA1 FA1 FA1 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 FA1 FA1 FA1 FA1 FA1 WA1 WA1 WA1 
WA1 WA1 FA1 FA1 FA1 FA1 FA1 FA1 FA1 WA1 WA1 
WA1 WA1 FA1 FA1 FA1 FA1 FA1 FA1 FA1 WA1 WA1 
WA1 WA1 FA1 FA1 FA1 FA1 FA1 FA1 FA1 WA1 WA1 
WA1 WA1 WA1 FA1 FA1 FA1 FA1 FA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 FA1 FA1 FA1 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA

# Reactor Dimensions & Housing
1. downcomer width 20 cm
2. vessel width 20 cm
3. Active core radii 73.80 cm
4. Whole core equivalent radii 141 cm
5. Core Height 240 cm
6. Active Core Hieght 200 cm
7. Bottom and Top Reflector Height 20 cm
8. Water Gap Downcomer to vessel 20 cm

In [43]:
width = 21.6038*8
height  = 240
radii = [0, 20, 40, 60]
mats = [clad, mod, clad]

h1 = housing(width=width, height=height, defaultCRFlag=True, radiiCR= radii, matsCR= mats)
hREF = h1.duplicateUniv("housingRef")


# Final Core Object and Settings
1. Power = 200E6 W
2. Assemblies 37
3. Active Core Height 200 cm

In [44]:
input1 = core(mainUniv=CORE1, housing=h1, lats=latList1, pins=pinList1, materials=matList, flagBurn=True, flagBranch=False, flagXS = False)
# input1.setXS(ngroups=2, ebounds=np.array([0.625E-6]), universes=[RB, RT], setFPPXS = True, setADF = False)
input1.setSettings(power=200E6, bc=[1, 1, 1], sym=[0], egrid = [5e-5, 1e-9, 15.0],  nps=50000, nact=80, nskip=20, setPCC = False, 
                   misc = ["ene 1 1 1E-11 0.625E-6 15"])

inputREF= core(mainUniv=CORE_REF, housing=hREF, lats=latlistRef, pins=pinList1, materials=matList, flagBurn=True, flagBranch=False, flagXS = False)
# input1.setXS(ngroups=2, ebounds=np.array([0.625E-6]), universes=[RB, RT], setFPPXS = True, setADF = False)
inputREF.setSettings(power=200E6, bc=[1, 1, 1], sym=[0], egrid = [5e-5, 1e-9, 15.0],  nps=50000, nact=80, nskip=20, setPCC = False, 
                   misc = ["ene 1 1 1E-11 0.625E-6 15"])

In [45]:
inventory = np.array([50100, 50110, 541350, 922340, 922350, 942390])
burnPoints = np.array([
0.1,
0.2,
0.3, 
0.4,  
0.5, 
0.6, 
0.7, 
0.8, 
0.9, 
1.0, 
2.0, 
3.0, 
4.0, 
5.0, 
6.0, 
7.0, 
8.0, 
9.0, 
10.0, 
11.0, 
12.0, 
13.0, 
14.0, 
15.0, 
16.0, 
17.0, 
18.0, 
19.0,
20.0, 
21.0, 
22.0, 
23.0, 
24.0, 
25.0, 
26.0, 
27.0, 
28.0, 
29.0, 
30.0, 
31.0, 
32.0
])
input1.setBurnup(inventory=inventory, burnPoints=burnPoints, isDayTot=False)
inputREF.setBurnup(inventory=inventory, burnPoints=burnPoints, isDayTot=False)

In [46]:
absPath = "/mnt/c/Users/iaguirre6/Documents/xsdata/endfb7"
thermLib = "therm lwtr lwj3.11t"
input1.setXSLib(absPath, thermLib)
inputREF.setXSLib(absPath, thermLib)

In [47]:
input1.writeFile("bigT3D_Simple.txt")
text_file = open("bigT3D_Simple.txt") #Display contents of created inputFile
file_content = text_file.read()
print(file_content)
text_file.close()


mat fuel1    -10.5216  burn 1  rgb 238 59 49
92235.09c	-0.04319337211
92238.09c	-0.83830401789
8016.09c	-0.11850261

mat fuel2    -10.5216  burn 1  rgb 255 165 0
92235.09c	-0.04319337211
92238.09c	-0.83830401789
8016.09c	-0.11850261

mat fuel3    -10.5216  burn 1  rgb 255 255 0
92235.09c	-0.04319337211
92238.09c	-0.83830401789
8016.09c	-0.11850261

mat fuel4    -10.5216  burn 1  rgb 255 255 255
92235.09c	-0.017629948
92238.09c	-0.863867442
8016.09c	-0.11850261

mat water    -7.2 moder H2O 1001   rgb 198 226 255
1001.06c	0.66667
8016.06c	0.33333

mat clad    -6.55    rgb 183 183 183
40090.06c	0.5145
40091.06c	0.1122
40092.06c	0.1715
40094.06c	0.1738
40096.06c	0.028

mat reflector    0.070462    rgb 204 153 255
40090.06c	0.0059311
40091.06c	0.0012934
40092.06c	0.001977
40094.06c	0.0020035
40096.06c	0.00032278
26054.06c	0.011217
26056.06c	0.17738
26057.06c	0.0042545
26058.06c	0.00054149
24050.06c	0.0024673
24052.06c	0.04758
24053.06c	0.0053951
24054.06c	0.001343
28058.06c	0.017122
28060.0